In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#いろいろインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#データセットの読み込み
train_df = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test_df = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
train_df['WhatIsData'] = 'Train'
test_df['WhatIsData'] = 'Test'
test_df['SalePrice'] = 9999999999

In [ ]:
# train_dataとtest_dataの連結
df = pd.concat([train_df,test_df],axis=0,sort=True).reset_index(drop=True)

In [ ]:
num2str_list = ['MSSubClass','YrSold','MoSold']
for column in num2str_list:
    df[column] = df[column].astype(str)

In [ ]:
for column in df.columns:
    if df[column].dtype=='O':
        df[column] = df[column].fillna('None')
    else:
        df[column] = df[column].fillna(0)

In [ ]:
df = pd.get_dummies(df)

In [ ]:
def divide_proc(all_data):
# マージデータを学習データとテストデータに分割
    train_ = df[df['WhatIsData']=='Train'].drop(['WhatIsData','Id'], axis=1).reset_index(drop=True)
    test_ = df[df['WhatIsData']=='Test'].drop(['WhatIsData','SalePrice'], axis=1).reset_index(drop=True)
# 学習データ内の分割
    train_x = train_.drop('SalePrice',axis=1)
    train_y = np.log(train_['SalePrice'])
# テストデータ内の分割
    test_id = test_['Id']
    test_data = test_.drop('Id',axis=1)
#
    return train_x,train_y,test_id,test_data

In [ ]:
#train_ = df[df['WhatIsData']=='Train'].drop(['WhatIsData','Id'], axis=1).reset_index(drop=True)
#test_ = df[df['WhatIsData']=='Test'].drop(['WhatIsData','SalePrice'], axis=1).reset_index(drop=True)

In [ ]:
#train_df = pd.merge(df.iloc[train_df.index[0]:train_df.index[-1]],train_df['SalePrice'],left_index=True,right_index=True)
#test_df = df.iloc[train_df.index[-1]:]
#train_ = train_df.drop(columns = ['SalePrice'])
#train_ = train_df
#test_ = test_df

In [ ]:
X = train_x[:,1:]  
y = train_y[:,0] 

In [ ]:
#X = train_.values[:,1:]  
#y = train_.values[:,0] 

In [ ]:
test_x = test_.values

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate

In [ ]:
select = SelectKBest(k = 300)




clf = RandomForestClassifier(random_state = 10, 
                             warm_start = True,  
                             n_estimators = 26,
                             max_depth = 6, 
                             max_features = 'sqrt')
pipeline = make_pipeline(select, clf)
pipeline.fit(X, y)

In [ ]:
#mask= select.get_support()

In [ ]:
#list_col = list(df.columns[1:])

In [ ]:
#for i, j in enumerate(list_col):
#    print('No'+str(i+1), j,'=',  mask[i])

In [ ]:
X_selected = select.transform(X)
print('X.shape={}, X_selected.shape={}'.format(X.shape, X_selected.shape))

In [ ]:
pred = pipeline.predict(test_x)

In [ ]:
print(pred)

In [ ]:
print(pred.shape)

In [ ]:
submission_df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv')


In [ ]:
submission_df.info

In [ ]:
submission_df['SalePrice'] = pred
submission_df.to_csv('submission.csv',index=False)